In [40]:
import pandas as pd
ames_housing = pd.read_csv("../datasets/house_prices.csv", na_values="?")
ames_housing = ames_housing.drop(columns="Id")

target_name = "SalePrice"
data, target = ames_housing.drop(columns=target_name), ames_housing[target_name]
target = (target > 200_000).astype(int)

In [2]:
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [3]:
target.iloc[:5]

0    1
1    0
2    1
3    0
4    1
Name: SalePrice, dtype: int64

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [4]:
data.shape

(1460, 79)

In [5]:
sum((data.dtypes == 'int64') | (data.dtypes == 'float64'))

36

In [6]:
data[['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt']]

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt
0,65.0,8450,7,5,2003
1,80.0,9600,6,8,1976
2,68.0,11250,7,5,2001
3,60.0,9550,7,5,1915
4,84.0,14260,8,5,2000
...,...,...,...,...,...
1455,62.0,7917,6,5,1999
1456,85.0,13175,6,6,1978
1457,66.0,9042,7,9,1941
1458,68.0,9717,5,6,1950


In [7]:
data[['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt']].describe()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt
count,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,70.049958,10516.828082,6.099315,5.575342,1971.267808
std,24.284752,9981.264932,1.382997,1.112799,30.202904
min,21.000000,1300.000000,1.000000,1.000000,1872.000000
25%,59.000000,7553.500000,5.000000,5.000000,1954.000000
50%,69.000000,9478.500000,6.000000,5.000000,1973.000000
75%,80.000000,11601.500000,7.000000,6.000000,2000.000000
max,313.000000,215245.000000,10.000000,9.000000,2010.000000


In [41]:
numerical_features = [
  "LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2",
  "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "LowQualFinSF",
  "GrLivArea", "BedroomAbvGr", "KitchenAbvGr", "TotRmsAbvGrd", "Fireplaces",
  "GarageCars", "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch",
  "3SsnPorch", "ScreenPorch", "PoolArea", "MiscVal",
]

In [42]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

scaler = StandardScaler()
imputer = SimpleImputer(strategy="mean")
model = LogisticRegression()

pipeline = make_pipeline(scaler, imputer, model)

In [43]:
pipeline.fit(data[numerical_features], target)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('simpleimputer', SimpleImputer()),
                ('logisticregression', LogisticRegression())])

In [44]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(pipeline, data[numerical_features], target, cv=5)
cv_results

{'fit_time': array([0.00954723, 0.00940895, 0.00931859, 0.00913501, 0.00989556]),
 'score_time': array([0.00163245, 0.00162721, 0.00164843, 0.00162101, 0.00164938]),
 'test_score': array([0.89383562, 0.9109589 , 0.89726027, 0.87671233, 0.89726027])}

In [45]:
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: ",
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is:  0.895 +/- 0.011


In [46]:
categorical_features = data.columns.difference(numerical_features)
len(categorical_features)

55

In [47]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder

imputer_categorical = SimpleImputer(strategy="most_frequent")

numerical_preprocessor = make_pipeline(scaler, imputer) # see previous block
categorical_preprocessor = make_pipeline(
    imputer_categorical,
    OneHotEncoder(handle_unknown="ignore")
)

preprocessor = ColumnTransformer([
    ('categorical', categorical_preprocessor, categorical_features),
    ('numerical', numerical_preprocessor, numerical_features)])

final_model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))

In [48]:
_ = final_model.fit(data, target)

In [37]:
cv_results2 = cross_validate(final_model, data, target, cv=5)

In [50]:
scores2 = cv_results2["test_score"]
print("The mean cross-validation accuracy is: ",
      f"{scores2.mean():.3f} +/- {scores2.std():.3f}")

The mean cross-validation accuracy is:  0.919 +/- 0.010


In [52]:
print("The difference is significant but is it substantial?",
      f"{scores2.mean() - scores.mean() > 3*scores2.std()}")

The difference is significant but is it substantial? False
